In [1]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
# %pip install -r requirements.txt
# %pip install python-docx pdfplumber
# %pip install sentence_transformers
# %pip install sentence-transformers
# %pip install --upgrade pip
# %pip install --no-cache-dir sentence-transformers

In [3]:
# %pip install torch torchvision torchaudio

In [4]:
import json
import pandas as pd
from gigasmol import GigaChatSmolModel
from smolagents import CodeAgent, ToolCallingAgent, Tool, UserInputTool

### Ключи api для доступа к GigaChat, сервисам погоды и курсом валют

Заполните в файле `credentials.json` соответствующие значения токенов. 

* https://www.exchangerate-api.com — сайт для получения актуальных курсов валют (получаем `currency_api_key`)
* https://openweathermap.org/api — сайт для получения актуальной информации о погоде (получаем `weather_api_key`)

In [5]:
credentials = json.load(open('credentials.json', 'r'))

In [6]:
credentials

{'gigachat_authorization_key': 'NTExMGY2MjQtYjgyNy00NThiLWJkZjMtODBiZWY4ZjdkMGZhOmVlYjFjMzhiLTlmODYtNDQwNi1hNzMwLWVjNzE4MTQwMWM0Ng==',
 'gigachat_client_id': '378bd664-11e3-46e2-899e-68516bdeeba1',
 'currency_api_key': '54598d4356b6418c82cdcb21',
 'weather_api_key': '908d33e3e53879e5f94dbdacaa701802',
 'news_api_key': '95ecc71971444bf8beacd1f6aec300b1'}

### Инициализация базовой модели

In [7]:
model = GigaChatSmolModel(
    auth_data=credentials['gigachat_authorization_key'],
    client_id=credentials['gigachat_client_id'],
    model_name="GigaChat-2-Max",
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1.1,
    max_tokens=64000
)

### Инициализация агентов

In [8]:
from tools.basic_tools import CurrencyConversionTool, TimeTool, NewsTool
from tools.utils import GigaChatFinalAnswerTool
from tools.contract_tools import ContractAnalyzerTool
from tools.regulation_tools import RegulationSearchTool

from pathlib import Path

In [9]:
from utils.logger import init_logging
init_logging()

In [10]:
agent = CodeAgent(
    tools=[
        CurrencyConversionTool(credentials['currency_api_key']),
        NewsTool(credentials['news_api_key']),
        ContractAnalyzerTool(model=model),
        RegulationSearchTool(model=model),
        TimeTool(),
        ],
    additional_authorized_imports=["datetime"],
    model=model
)

agent.tools['final_answer'] = GigaChatFinalAnswerTool()

agent.system_prompt = (
    Path("prompts/system_prompt.md").read_text(encoding="utf-8")
    + "\n\n"
    + Path("prompts/examples.md").read_text(encoding="utf-8")
)

[RegulationSearchTool] Загружаю документы (упрощенный режим)...
✅ Загружено фрагментов: 1041


### Интерфейс агента

In [11]:
from ui.agent_ui import GradioUI

In [12]:
# # Можно вести диалог с агентом из jupyter notebook или через интерфейс
# agent.run('Какая погода в спб, нск и москве?', reset=False)
# agent.run('Какой курс доллара к рублю', reset=False)
# agent.run('Какие последние новости в сфере финансов?', reset=False)

# agent.run('Проанализируй следующий документ: ' + doc, reset=False)

In [13]:
agent.run("Могу ли я вернуть деньги за страховку по кредиту?", reset=False)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Могу ли я вернуть деньги за страховку по кредиту?                                                               │
│                                                                                                                 │
╰─ GigaChatSmolModel -  ──────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  regulation_info = regulation_search(query='возврат страховой суммы по кредиту')                                  
  print(regulation_info)                                                                                           
                                                                                                                   
  final_answer('Да, вы можете вернуть часть средств за страховку по кредиту, но это зависит от условий вашего      
  конкретного договора и политики вашей страховой компании.')                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

ERROR:root:Critical error in __call__: string indices must be integers, not 'str'
Traceback (most recent call last):
  File "c:\Users\gorevoy0220\AppData\Local\Programs\Python\Python312\Lib\site-packages\gigasmol\models.py", line 271, in __call__
    messages = map_message_roles_to_api_format(messages)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gorevoy0220\AppData\Local\Programs\Python\Python312\Lib\site-packages\gigasmol\models.py", line 130, in map_message_roles_to_api_format
    message_role = TOOL_ROLE_CONVERSIONS[message['role']]
                                         ~~~~~~~^^^^^^^^
TypeError: string indices must be integers, not 'str'


Execution logs:
Error in model execution: string indices must be integers, not 'str'

Out - Final answer: Да, вы можете вернуть часть средств за страховку по кредиту, но это зависит от условий вашего 
конкретного договора и политики вашей страховой компании.

[Step 1: Duration 3.99 seconds]

'Да, вы можете вернуть часть средств за страховку по кредиту, но это зависит от условий вашего конкретного договора и политики вашей страховой компании.'

In [14]:
# hide_steps=True — скрывает шаги в интерфейсе, но выводит их ниже в jupyter notebook 
GradioUI(agent, mailbox=None, calendar=None, hide_steps=False).launch()

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f36d4e7c28def08b89.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Привет!                                                                                                         │
│                                                                                                                 │
╰─ GigaChatSmolModel -  ──────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer('Здравствуйте!')                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Здравствуйте!

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer('Я могу помогать вам с различными задачами: отвечать на вопросы, анализировать тексты договоров и   
  страховых полисов, искать информацию в нормативных актах, предоставлять актуальные курсы валют и многое другое.  
  Расскажите подробнее, какая помощь нужна именно сейчас.')                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Я могу помогать вам с различными задачами: отвечать на вопросы, анализировать тексты договоров 
и страховых полисов, искать информацию в нормативных актах, предоставлять актуальные курсы валют и многое другое. 
Расскажите подробнее, какая помощь нужна именно сейчас.

[Step 4: Duration 2.80 seconds]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f36d4e7c28def08b89.gradio.live


In [ ]:
agent.run("Сколько стоит 100 долларов в рублях?")

In [ ]:
from core.runner import run_and_log

DEPOSIT_TEXT = """
Договор банковского депозита
1. Срок размещения средств составляет 12 месяцев.
2. Процентная ставка составляет 9% годовых.
3. В случае досрочного расторжения применяется ставка рефинансирования.
"""

task = f"Проанализируй следующий текст договора:\n\n{DEPOSIT_TEXT}"
result = run_and_log(agent, task)

In [ ]:
df = pd.read_csv("logs/agent_calls.csv", encoding="utf-8-sig")

print("📊 Всего выполнено задач:", len(df))

print("\n🛠️ Используемые инструменты:")
print(df["tool_used"].value_counts())

print("\n📈 Среднее время ответа (сек):")
print(df["step_duration_sec"].dropna().mean())

print("\n📚 Примеры запросов:")
print(df[["user_input", "tool_used", "final_answer"]].tail(3).to_string(index=False))

In [ ]:
df

In [ ]:
from scripts.evaluate_agent import load_eval_tasks

tasks = load_eval_tasks("docs/eval_tasks.jsonl")
print(f"📋 Загружено задач: {len(tasks)}")

# Мини-вывод на экран
for t in tasks[:3]:
    print(t["task"])

In [ ]:
from scripts.evaluate_agent import load_eval_tasks, evaluate, save_results, print_summary
from core.runner import run_and_log

# Загружаем задачи
tasks = load_eval_tasks("docs/eval_tasks.jsonl")

# Прогоняем через агента и run_and_log
results = evaluate(agent, tasks, run_and_log)

# Сохраняем или просто смотрим
save_results(results, "logs/eval_results.jsonl")  # или не сохранять
print_summary(results)

# Посмотреть результаты
pd.DataFrame(results)